# TRAINING I3W


# A) Create Envorinment, Vehicles etc

### General Parameter

In [1]:
# Define horizon as a variable to ensure consistent use across notebook (length of one rollout)
HORIZON=500                                 #103 max Horizon, wenn es vor verlassen abbrechen soll!, default war 500

# name of the experiment
experiment_name = "IntersectionExample"

# scenario class
import flow.scenarios as scenarios
print("Available scenarios:")
print(scenarios.__all__)
scenario_name = "IntersectionTWScenario"

# environment class
import flow.multiagent_envs as flowenvs
print("\nAvailable environments:")
print(flowenvs.__all__)
env_name = "MultiAgentIntersectionEnv"

Available scenarios:
['Scenario', 'BayBridgeScenario', 'BayBridgeTollScenario', 'BottleneckScenario', 'Figure8Scenario', 'SimpleGridScenario', 'HighwayScenario', 'LoopScenario', 'MergeScenario', 'TwoLoopsOneMergingScenario', 'MultiLoopScenario', 'IntersectionScenarioTW']

Available environments:
['MultiEnv', 'MultiAgentAccelEnv', 'MultiWaveAttenuationPOEnv', 'MultiAgentIntersectionEnv', 'MultiAgentTeamSpiritIntersectionEnv', 'MultiAgentIntersectionEnv_baseline_1', 'MultiAgentIntersectionEnv_baseline_2', 'MultiAgentIntersectionEnv_baseline_3']


### Net Parameter

In [2]:
from flow.core.params import NetParams
from flow.scenarios.intersection import ADDITIONAL_NET_PARAMS

additionalNetParams = {
            "edge_length": 40,
            "lanes": 1,
            "speed_limit": 30
        }

net_params = NetParams( no_internal_links=False,                  #default: True   !! damit Kreuzungen nicht überspr. werden
                        inflows=None,                             #default: None
                        osm_path=None,                            #default: None
                        netfile=None,                             #default: None
                        additional_params=additionalNetParams     #default: None   !!
                      )

### InitialConfig Parameter

In [3]:
from flow.core.params import InitialConfig

initial_config = InitialConfig( shuffle=True,                            #default: False         !!
                                spacing="custom",                        #default: "uniform"     !!
                                min_gap=10,                              #default: 0
                                perturbation=29.99,                      #default: 0.0            !!        
                                x0=0,                                    #default: 0
                                bunching=0,                              #default: 0
                                lanes_distribution=float("inf"),         #default: float("inf")
                                edges_distribution="all",                #default: "all"
                                additional_params=None )                 #default: None

### SUMO Parameter

In [4]:
from flow.core.params import SumoParams

sumo_params = SumoParams( port = None,                  #default: None
                          sim_step=0.1,                 #default: 0.1
                          emission_path=None,           #default: None
                          lateral_resolution=None,      #default: None
                          no_step_log=True,             #default: True
                          render=False,                 #default: False
                          save_render=False,            #default: False
                          sight_radius=25,              #default: 25
                          show_radius=False,            #default: False
                          pxpm=2,                       #default: 2
                          overtake_right=False,         #default: False    
                          seed=None,                    #default: None
                          restart_instance=False,       #default: False
                          print_warnings=True,          #default: True
                          teleport_time=-1,             #default: -1
                          num_clients=1,                #default: 1
                          sumo_binary=None )            #default: None

### Environment Parameter

In [5]:
from flow.core.params import EnvParams

additionalEnvParams = {
        # maximum acceleration of autonomous vehicles
        "max_accel": 3,
        # maximum deceleration of autonomous vehicles
        "max_decel": 3,
        "target_velocity": 30
    }

env_params = EnvParams( additional_params=additionalEnvParams, #default: None    !!
                        horizon=HORIZON,                       #default: 500     !!
                        warmup_steps=0,                        #default: 0       
                        sims_per_step=1,                       #default: 1
                        evaluate=False )                       #default: False

### Vehicles Parameter

In [6]:
from flow.core.params import VehicleParams

# import vehicles dynamics models
#from flow.controllers import SumoCarFollowingController
from flow.controllers import ContinuousRouter
#from flow.controllers.lane_change_controllers import SumoLaneChangeController
from flow.controllers.lane_change_controllers import StaticLaneChanger
from flow.controllers import RLController
from flow.core.params import SumoLaneChangeParams
from flow.core.params import SumoCarFollowingParams
from random import *

vehicles = VehicleParams()

#### Add RL-Agent controlled vehicles 

In [7]:
# car following parameters, default: None
cf_parameter = SumoCarFollowingParams(
                speed_mode="aggressive")
# lane change parameters, default: None
lc_parameter =  None

vehicles.add( # name of the vehicle
                veh_id = "rl",
              # acceleration controller, default: (SumoCarFollowingController, {})
                acceleration_controller=(RLController, {}),
              # lane_change_controller, default: (SumoLaneChangeController, {})
                lane_change_controller=(StaticLaneChanger,{}),
              # routing controller, default: None
                routing_controller=(ContinuousRouter, {}),
              # initial speed, default: 0
                initial_speed=0,
              # number of vehicles, default: 1 
                num_vehicles=2,
                
                car_following_params=cf_parameter
              # speed mode, default: "right_of_way"
                #speed_mode="aggressive",
              # lane change mode, default: "no_lat_collide"
                #lane_change_mode="aggressive", 
              # car following parameter, default: None
                #sumo_car_following_params=cf_parameter,
              # lane change parameter, default: None
                #sumo_lc_params=lc_parameter
)

### Flow Parameter

In [8]:
# Creating flow_params. Make sure the dictionary keys are as specified. 
flow_params = dict( # name of the experiment
                      exp_tag=experiment_name,
                    # name of the flow environment the experiment is running on
                      env_name=env_name,
                    # name of the scenario class the experiment uses
                      scenario=scenario_name,
                    # simulator that is used by the experiment
                      simulator='traci',
                    # sumo-related parameters (see flow.core.params.SumoParams)
                      sim=sumo_params,
                    # environment related parameters (see flow.core.params.EnvParams)
                      env=env_params,
                    # network-related parameters (see flow.core.params.NetParams and
                    # the scenario's documentation or ADDITIONAL_NET_PARAMS component)
                      net=net_params,
                    # vehicles to be placed in the network at the start of a rollout 
                    # (see flow.core.vehicles.Vehicles)
                      veh=vehicles,
                   # (optional) parameters affecting the positioning of vehicles upon 
                   # initialization/reset (see flow.core.params.InitialConfig)
                      initial=initial_config
                )

# B) Training

In [9]:
import json

import ray
try:
    from ray.rllib.agents.agent import get_agent_class
except ImportError:
    from ray.rllib.agents.registry import get_agent_class
from ray.tune import run_experiments
from ray.tune.registry import register_env

from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder

from ray import tune
from ray.rllib.agents.ppo.ppo_policy_graph import PPOPolicyGraph

In [10]:
# number of parallel workers
N_CPUS = 2
# number of rollouts per training iteration
N_ROLLOUTS = 20

ray.init(redirect_output=True, num_cpus=N_CPUS+1)

Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-03-20_23-42-41_12635/logs.
Waiting for redis server at 127.0.0.1:53923 to respond...
Waiting for redis server at 127.0.0.1:47681 to respond...
Starting the Plasma object store with 6.554658406 GB memory using /dev/shm.

View the web UI at http://localhost:8889/notebooks/ray_ui.ipynb?token=a0a144a5bab3e38edf499afd28254d461f3b525382fe70af



{'node_ip_address': '192.168.2.102',
 'object_store_addresses': ['/tmp/ray/session_2019-03-20_23-42-41_12635/sockets/plasma_store'],
 'raylet_socket_names': ['/tmp/ray/session_2019-03-20_23-42-41_12635/sockets/raylet'],
 'redis_address': '192.168.2.102:53923',
 'webui_url': 'http://localhost:8889/notebooks/ray_ui.ipynb?token=a0a144a5bab3e38edf499afd28254d461f3b525382fe70af'}

In [11]:
# The algorithm or model to train. This may refer to "
#      "the name of a built-on algorithm (e.g. RLLib's DQN "
#      "or PPO), or a user-defined trainable function or "
#      "class registered in the tune registry.")
alg_run = "PPO"

agent_cls = get_agent_class(alg_run)
config = agent_cls._default_config.copy()
config["num_workers"] = N_CPUS  # number of parallel workers
config["train_batch_size"] = HORIZON * N_ROLLOUTS  # batch size
config["gamma"] = 0.9  # discount rate default 0.999
config["model"].update({"fcnet_hiddens": [100, 50, 25]})  # size of hidden layers in network defaule 64 32
config["use_gae"] = True  # using generalized advantage estimation
config["lambda"] = 0.97  
#config["sgd_minibatch_size"] = min(16 * 1024, config["train_batch_size"])  # stochastic gradient descent
#config["sample_batch_size"] = config["train_batch_size"]/config["num_workers"] # 200 default, trotzdem zu hoch?
config["kl_target"] = 0.02  # target KL divergence
config["num_sgd_iter"] = 10  # number of SGD iterations
config["horizon"] = HORIZON  # rollout horizon

# save the flow params for replay
flow_json = json.dumps(flow_params, cls=FlowParamsEncoder, sort_keys=True,
                       indent=4)  # generating a string version of flow_params
config['env_config']['flow_params'] = flow_json  # adding the flow_params to config dict
config['env_config']['run'] = alg_run

# Call the utility function make_create_env to be able to 
# register the Flow env for this experiment
create_env, gym_name = make_create_env(params=flow_params, version=0)

# Register as rllib env with Gym
register_env(gym_name, create_env)

In [12]:
# multi agent policy mapping
test_env = create_env()
obs_space = test_env.observation_space
act_space = test_env.action_space

def gen_policy():
    return (PPOPolicyGraph, obs_space, act_space, {})

# Setup PG with an ensemble of `num_policies` different policy graphs
policy_graphs = {'rl_0': gen_policy()}
    
def policy_mapping_fn(agent_id):
    return 'rl_0'

config.update({
        'multiagent': {
            'policy_graphs': policy_graphs,
            'policy_mapping_fn': tune.function(policy_mapping_fn),
            'policies_to_train': ['rl_0']
        }
    })

 Starting SUMO on port 35979


25.810823812562866
19.209701566894147


In [13]:
trials = run_experiments({
    flow_params["exp_tag"]: {
        "run": alg_run,  # RL algorithm to run
        "env": gym_name,  # environment name generated earlier
        "config": {  # configuration params (must match "run" value)
            **config
        },
        "checkpoint_freq": 1,  # number of iterations between checkpoints
        "max_failures": 999,
        "stop": {  # stopping conditions
            "training_iteration": 1000,  # number of iterations to stop after
        },
    },
})

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/3 CPUs, 0/1 GPUs
Memory usage on this node: 3.9/16.4 GB

Created LogSyncer for /home/thorsten/ray_results/IntersectionExample/PPO_MultiAgentIntersectionEnv-v0_0_2019-03-20_23-42-430ht1kvt9 -> 
== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 3.9/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-20_23-43-47
  done: false
  episode_len_mean: 485.3
  episode_reward_max: 260.1386717794316
  episode_reward_mean: 104.50339380636703
  episode_reward_min: -135.2918705150996
  episodes_this_iter: 20
  episodes_total: 20
  experiment_id: d110021820ee4abd9ffac6de3e7266c0
  hostname: Gandalf
  info:
    grad_time_ms: 4752.245
    load_time_ms: 46.154
    num_steps_sampled: 10000
    num_steps_trained: 

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-20_23-45-49
  done: false
  episode_len_mean: 276.81
  episode_reward_max: 376.0977609287789
  episode_reward_mean: 113.25592566381307
  episode_reward_min: -165.7508589951873
  episodes_this_iter: 32
  episodes_total: 175
  experiment_id: d110021820ee4abd9ffac6de3e7266c0
  hostname: Gandalf
  info:
    grad_time_ms: 4293.955
    load_time_ms: 9.186
    num_steps_sampled: 60000
    num_steps_trained: 60000
    rl_0:
      cur_kl_coeff: 0.006250001490116119
      cur_lr: 4.999999873689376e-05
      entropy: 1.413277268409729
      kl: 0.0043801916763186455
      policy_loss: -0.0019741083960980177
      total_loss: 34.99400329589844
      vf_explained_var: 0.4350240230560303
      vf_loss: 34.995948791503906
    sample_time_ms: 21132.968
    update_time_ms: 119.305
  iterations_since_restore: 6
  node_ip: 192.168.2.102
  num_metric_batches_dropped: 0
  pid: 12721
  policy_reward_mean:
    rl_0: 56.627962

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 4.8/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING [pid=12721], 267 s, 11 iter, 110000 ts, 114 rew

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-20_23-48-07
  done: false
  episode_len_mean: 190.76
  episode_reward_max: 347.43331604544795
  episode_reward_mean: 111.15970060103659
  episode_reward_min: -165.23983642176515
  episodes_this_iter: 54
  episodes_total: 456
  experiment_id: d110021820ee4abd9ffac6de3e7266c0
  hostname: Gandalf
  info:
    grad_time_ms: 4039.984
    load_time_ms: 1.668
    num_steps_sampled: 120000
    num_steps_trained: 120000
    rl_0:
      cur_kl_coeff: 0.00019531254656612873
      cur_lr: 4.999999873689376e-05
      entropy: 1.3768444061279297
      kl: 0.002570231445133686
      policy_loss: -0.0011611907975748181
      t

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 4.7/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING [pid=12721], 407 s, 17 iter, 170000 ts, 162 rew

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-20_23-50-27
  done: false
  episode_len_mean: 138.9
  episode_reward_max: 332.6232781298981
  episode_reward_mean: 185.5375352776726
  episode_reward_min: -163.5594249847785
  episodes_this_iter: 74
  episodes_total: 842
  experiment_id: d110021820ee4abd9ffac6de3e7266c0
  hostname: Gandalf
  info:
    grad_time_ms: 4010.697
    load_time_ms: 1.662
    num_steps_sampled: 180000
    num_steps_trained: 180000
    rl_0:
      cur_kl_coeff: 2.441406832076609e-05
      cur_lr: 4.999999873689376e-05
      entropy: 1.3100078105926514
      kl: 0.010581749491393566
      policy_loss: -0.0021600350737571716
      total_

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 4.7/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING [pid=12721], 544 s, 23 iter, 230000 ts, 160 rew

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-20_23-52-44
  done: false
  episode_len_mean: 118.07
  episode_reward_max: 348.85261408596284
  episode_reward_mean: 206.35341746696548
  episode_reward_min: -162.05121677224912
  episodes_this_iter: 83
  episodes_total: 1331
  experiment_id: d110021820ee4abd9ffac6de3e7266c0
  hostname: Gandalf
  info:
    grad_time_ms: 3947.411
    load_time_ms: 1.593
    num_steps_sampled: 240000
    num_steps_trained: 240000
    rl_0:
      cur_kl_coeff: 6.103517080191523e-06
      cur_lr: 4.999999873689376e-05
      entropy: 1.2192636728286743
      kl: 0.011002766899764538
      policy_loss: -0.0031391833908855915
      t

...

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-21_05-41-13
  done: false
  episode_len_mean: 78.2734375
  episode_reward_max: 393.6716915941062
  episode_reward_mean: 89.34154047660667
  episode_reward_min: -162.97310084494592
  episodes_this_iter: 128
  episodes_total: 123797
  experiment_id: d110021820ee4abd9ffac6de3e7266c0
  hostname: Gandalf
  info:
    grad_time_ms: 3696.574
    load_time_ms: 1.526
    num_steps_sampled: 9760000
    num_steps_trained: 9760000
    rl_0:
      cur_kl_coeff: 0.10041848570108414
      cur_lr: 4.999999873689376e-05
      entropy: -0.5630508661270142
      kl: 0.017911629751324654
      policy_loss: 0.006028824020177126
      total_loss: 43.29407501220703
      vf_explained_var: 0.9028006196022034
      vf_loss: 43.286251068115234
    sample_time_ms: 17921.944
    update_time_ms: 4.834
  iterations_since_restore: 976
  node_ip: 192.168.2.102
  num_metric_batches_dropped: 0
  pid: 12721
  policy_reward_mean:
    rl_0:

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 8.0/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING [pid=12721], 21548 s, 981 iter, 9810000 ts, 118 rew

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-21_05-43-24
  done: false
  episode_len_mean: 80.76612903225806
  episode_reward_max: 388.25657571649805
  episode_reward_mean: 113.09439791993184
  episode_reward_min: -168.6737335123682
  episodes_this_iter: 124
  episodes_total: 124545
  experiment_id: d110021820ee4abd9ffac6de3e7266c0
  hostname: Gandalf
  info:
    grad_time_ms: 3689.245
    load_time_ms: 1.448
    num_steps_sampled: 9820000
    num_steps_trained: 9820000
    rl_0:
      cur_kl_coeff: 0.10041848570108414
      cur_lr: 4.999999873689376e-05
      entropy: -0.5172299146652222
      kl: 0.022144366055727005
      policy_loss: 0.0055385124

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-21_05-45-14
  done: false
  episode_len_mean: 84.45299145299145
  episode_reward_max: 385.73578708079395
  episode_reward_mean: 133.8345763581821
  episode_reward_min: -160.4611939071083
  episodes_this_iter: 117
  episodes_total: 125160
  experiment_id: d110021820ee4abd9ffac6de3e7266c0
  hostname: Gandalf
  info:
    grad_time_ms: 3680.385
    load_time_ms: 1.604
    num_steps_sampled: 9870000
    num_steps_trained: 9870000
    rl_0:
      cur_kl_coeff: 0.10041848570108414
      cur_lr: 4.999999873689376e-05
      entropy: -0.5269858241081238
      kl: 0.015564343892037868
      policy_loss: 0.003117077983915806
      total_loss: 38.76171112060547
      vf_explained_var: 0.8970364332199097
      vf_loss: 38.75703048706055
    sample_time_ms: 18173.204
    update_time_ms: 5.165
  iterations_since_restore: 987
  node_ip: 192.168.2.102
  num_metric_batches_dropped: 0
  pid: 12721
  policy_reward_mean:
   

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 8.1/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING [pid=12721], 21788 s, 992 iter, 9920000 ts, 92.9 rew

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-21_05-47-26
  done: false
  episode_len_mean: 81.00813008130082
  episode_reward_max: 388.83289275340127
  episode_reward_mean: 111.22601706087224
  episode_reward_min: -164.71392959388731
  episodes_this_iter: 123
  episodes_total: 125903
  experiment_id: d110021820ee4abd9ffac6de3e7266c0
  hostname: Gandalf
  info:
    grad_time_ms: 3678.499
    load_time_ms: 1.51
    num_steps_sampled: 9930000
    num_steps_trained: 9930000
    rl_0:
      cur_kl_coeff: 0.10041848570108414
      cur_lr: 4.999999873689376e-05
      entropy: -0.5443569421768188
      kl: 0.019891388714313507
      policy_loss: 0.004040536

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-21_05-49-14
  done: false
  episode_len_mean: 81.91803278688525
  episode_reward_max: 381.78698660900255
  episode_reward_mean: 116.56326085406232
  episode_reward_min: -166.6732580035591
  episodes_this_iter: 122
  episodes_total: 126517
  experiment_id: d110021820ee4abd9ffac6de3e7266c0
  hostname: Gandalf
  info:
    grad_time_ms: 3687.746
    load_time_ms: 1.497
    num_steps_sampled: 9980000
    num_steps_trained: 9980000
    rl_0:
      cur_kl_coeff: 0.10041848570108414
      cur_lr: 4.999999873689376e-05
      entropy: -0.5273405313491821
      kl: 0.01697000488638878
      policy_loss: 0.0053238580003380775
      total_loss: 38.95095443725586
      vf_explained_var: 0.9049514532089233
      vf_loss: 38.94392776489258
    sample_time_ms: 18192.862
    update_time_ms: 4.908
  iterations_since_restore: 998
  node_ip: 192.168.2.102
  num_metric_batches_dropped: 0
  pid: 12721
  policy_reward_mean:
  